In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

In [2]:
circ = 26700 # m
c    = 3e8 # m / s
mu_tau = 2.2e-6 # s

n_steps = 1000

e_max = 5000 # GeV
e_inj = 400 
e_per_turn = 14

def f(e_inj, e_per_turn):
    
    # Calculate things in terms of time
    t_per_turn = circ / c
    e_per_sec = e_per_turn / t_per_turn
    t_max = (e_max - e_inj)/e_per_sec
    
    # Create plot of energy v. time
    t_vals = np.linspace(0, t_max, n_steps)
    e_vals = t_vals*e_per_sec + e_inj
    plt.plot(t_vals, e_vals)
    plt.xlabel("Time [s]")
    plt.ylabel("Energy [GeV]")
    plt.ylim(bottom=0)
    plt.grid()
    plt.show()
    
    # Create plot of decay fraction v. time
    t_step = t_max / n_steps
    g_vals = e_vals / .106
    t_step_proper_vals = t_step / g_vals
    decay_frac_vals = np.exp(-t_step_proper_vals/mu_tau)
    remaining_vals = []
    for i, val in enumerate(decay_frac_vals):
        if i == 0:
            remaining_vals.append(1)
        else:
            remaining_vals.append(val*remaining_vals[i-1])
    plt.plot(t_vals, remaining_vals)
    plt.xlabel("Time [s]")
    plt.ylabel("Fraction of beam remaining")
    plt.ylim(bottom=0)
    plt.grid()
    plt.show()
    
    # Also plot v. energy
    plt.plot(e_vals, remaining_vals)
    plt.xlabel("Energy [GeV]")
    plt.ylabel("Fraction of beam remaining")
    plt.ylim(bottom=0)
    plt.grid()
    plt.show()
     

In [3]:
interact(f, e_inj=e_inj, e_per_turn=widgets.IntSlider(min=1, max=50, value=e_per_turn))

interactive(children=(IntSlider(value=400, description='e_inj', max=1200, min=-400), IntSlider(value=14, descr…

<function __main__.f(e_inj, e_per_turn)>